# Toolbox Convention ("tbx")

The default convention shipped with the `h5rdmtoolbox` is called "tbx" and is largely based on the CF Metadata Convention (https://cfconventions.org/).

It enforces the usage of various attributes for the root group, groups in general or datasets. Some of them are required during object creation, thus they become available during calls like `create_group` or `create_dataset`.

For the **root group**, so during **file creation** the following attributes are requested (Root/File attribute are optional as they would be required also during read-mode. In future, there will be an option to make it required only during specific modes):
- `title`: Brief description of file content
- `institution` (optional): Link or address of institution
- `references` (optional): Links related to the data
- `standard_name_table` (optional): Standard name table used for this file
- `comment` (optional): User comment
- `contact` (optional): One or multiple users as ORCID IDs

For **group** creation:
- `comment` (optional): Additional user comment
- `contact` (optional): User(s) as ORCID ID(s) responsible or involved for the group content

For **dataset** creation:
- `standard_name`: Standardized name listed in Standard Name Table to describe the dataset
- `units`: Physical unit of the dataset
- `long_name` (optional): A human-readable succinct description of the group content
- `comment` (optional): User comment, providing extra information.
- `contact` (optional): User(s) as ORCID ID(s) responsible/involved for/in the dataset
- `source` (optional): Source of the data (Hardware or Software description)

At its core, so-called `standard_names` are defined for each relevant physical quantity. It is documented with its canonical units and a comprehensive description in a versioned table available to all users. No credits are taken for standardized attributes taken from the CF Metadata Convention. However, some attributes are slightly modified or used differntly.

In [1]:
import h5rdmtoolbox as h5tbx

To check, which attributes are available for which HDF5object (`File`, `Dataset`, `Group`), call `h5tbx.conventions.registered_conventions['tbx']`:

In [2]:
h5tbx.conventions.registered_conventions['tbx']

Convention(tbx)
> Properties:
File:
    * title: TitleAttribute
    * institution: any_string
    * references: ReferencesAttribute
    * standard_name_table: StandardNameTableAttribute
    * comment: CommentAttribute
    * contact: ContactAttribute
Dataset:
    * standard_name_table: StandardNameTableAttribute
    * standard_name: StandardNameAttribute
    * units: UnitsAttribute
    * long_name: LongNameAttribute
    * comment: CommentAttribute
    * contact: ContactAttribute
    * source: SourceAttribute
Group:
    * comment: CommentAttribute
    * contact: ContactAttribute
> Methods:
  init_file:
    * title (opt=None)
    * institution (opt=None)
    * references (opt=None)
    * standard_name_table (opt=None)
    * comment (opt=None)
    * contact (opt=None)
  create_group:
    * comment (opt=None)
    * contact (opt=None)
  create_dataset:
    * standard_name
    * units
    * long_name
    * comment (opt=None)
    * contact (opt=None)
    * source (opt=None)
  create_string_dat

## Standard Names
The `standard_name` is used to identify a physical quantity. To fulfill interoperability, findability as well as
re-usability each dataset **must** be assigned with either the attribute `long_name` or `standard_name`.

long_name
    A human-readable string.
standard_name
    A string respecStandardized Name Table
-----------------------
A standardized name table (snt) again is motivated by the CF Metadata Convention. It is a table
containing at least name, description and canonical_units. A python class is provided to read
a table from and write to a YML or XML file. Such an object
is passed to a wrapper-HDF-class to control the above described metadata of datasets.ting more or less strict rules defined by a community and defined in a name table.

Note, that the `long_name` attribute does not guarantee interoperability but `standard_name` does, if
the convention is known to each user.
In addition to this, an additional attribute is required, namely `units`. As we work with scientific
data, each dataset has a physical unit, e.g. [m]. If no physical unit can be set, it might because the
variable is dimensionless, which is an information about the unit anyhow, so we set `units=''`.

If the `standard_name` is provided in the dataset creation method and a standard name table (snt) is
available, then `units` is verified by that table. The table holds the base-units (canonical units) for
each standard name (check is performed on basic SI-units).


In [3]:
from h5rdmtoolbox.conventions import StandardName
StandardName(name='x_wind', canonical_units='m/s', description='x wind component')

<StandardName: x_wind [m/s] | SNT: None | desc: x wind component>

## Standardized Name Table

A standardized name table (snt) again is motivated by the CF Metadata Convention. It is a table
containing at least name, description and canonical_units. A python class is provided to read
a table from and write to a YML or XML file. Such an object
is passed to a wrapper-HDF-class to control the above described metadata of datasets.

## Usage

To activate the convention inside the `h5rdmtoolbox`, call `use(tbx)`:

In [4]:
h5tbx.use('tbx')

Whenever a dataset is written and the parameter `standard_name` is set, it is verified against the standard name convention/table associated with the wrapper class. If the constant `STRICT` is set to True (default), the name is looked-up in the table and, if not found, the dataset cannot be written. To allow standard names, that fulfill the spelling requirements but are not yet listed in the table, set `STRICT` to False:

In [5]:
h5tbx.conventions.standard_name.STRICT = False

The string representation of the HDF5 file object indicates, that the tbx-convention is enabled:

Some attributes are obligatory to provide with a dataset:
- `units`
- `standard_name`
- `long_name` (not needed if `standard_name` exists and vise versa)

**units**<br>
We expect that each data set written to the HDF5 file has a physical unit or no unit at all. It is registered in the attribute `units`.

**standard_name and long_name**<br>
For the sake of improved readability and interpretability the convention suggests to use `long_name` or `standard_name` as additional attributes. While `long_name` is human-readable and interpretable attribute, `standard_name` is intended to be read by a machine (other software). This allows to automate exploration and processing work.

### Dataset Creation

In [6]:
from h5rdmtoolbox import tutorial
snt = tutorial.get_standard_name_table()

with h5tbx.File(title='MyFile', standard_name_table=snt) as h5:
    h5.create_dataset('u', data=1.2,
                      units='m/s',
                      standard_name='x_velocity')
    h5.create_dataset('v', data=[1,2,3],
                      units='m/s',
                      standard_name='y_velocity')
    h5.create_dataset('method', data="linear_interpolation",
                      long_name='The inteprolation method used for something.')
    h5.dump()

C:\Users\da4323\Documents\programming\GitHub\h5RDMtoolbox\h5rdmtoolbox\conventions\standard_name.py:1206: UserWarning: No URL provided. Will save as dictionary.
  warnings.warn(f'No URL provided. Will save as dictionary.')


## Group creation

In addition to the standard way of creating groups, a `long_name` can be passed. It is optional, though:

In [7]:
with h5tbx.File() as h5:
    h5.create_group('mygrp')
    h5.create_group('othergrp', long_name='my other group')
    h5.sdump()

a: __h5rdmtoolbox_version__: 0.5.1a1
/mygrp
/othergrp


### Other attributes definitions

During file initialization, a `title`:

In [8]:
with h5tbx.File(title='MyFile') as h5:
    h5.dump()

## Initialize a Standard Name Convention
A standardized name table is a XML document, which contains (at least) a description and a canonical unit for a standarized name. We'll build one from scratch first and then have a look into already implemented ones:

Call `StandardNameTable` from the sub-package `conventions` and provide a `name`, `version`, `table`, `contact` and and `insitution`:

In [9]:
from h5rdmtoolbox.conventions import StandardNameTable

In [10]:
sc = StandardNameTable(
    name='Test_SNC',
    table={},
    version_number=1,
    contact='contact@python.com',
    institution='my_institution'
)
sc

2023-05-06_19:14:46,911 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com
2023-05-06_19:14:46,911 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com


Test_SNC (version number: 1)

We have built an empty convention (no table content). Lets add content. We can do this by creating a dictionary first...

In [11]:
tabledict = {'x_velocity': dict(canonical_units='m/s', description='velocity is a vector quantity.')}
tabledict

{'x_velocity': {'canonical_units': 'm/s',
  'description': 'velocity is a vector quantity.'}}

... and add it to the object by calling `update()`:

In [12]:
sc.update(tabledict)
sc.dump()

,canonical_units,description
x_velocity,m/s,velocity is a vector quantity.


New entries can be assigned by using `set` or `modified` depending on whether the entry already exists or not:

In [13]:
sc.set('time', canonical_units='s', description='physical time')
sc.modify('x_velocity', canonical_units='m/s', description='velocity is a vector quantity. x indicates the component in y-axis direction')
sc.set('y_velocity', canonical_units='m/s', description='velocity is a vector quantity. y indicates the component in y-axis direction')
sc.set('z_velocity', canonical_units='m/s', description='velocity is a vector quantity. z indicates the component in z-axis direction')
sc.sdump()

Test_SNC (version: 1)
+------------+-------------------+------------------------------------------------------------------------------+
|            | canonical_units   | description                                                                  |
|------------+-------------------+------------------------------------------------------------------------------|
| time       | s                 | physical time                                                                |
| x_velocity | m/s               | velocity is a vector quantity. x indicates the component in y-axis direction |
| y_velocity | m/s               | velocity is a vector quantity. y indicates the component in y-axis direction |
| z_velocity | m/s               | velocity is a vector quantity. z indicates the component in z-axis direction |
+------------+-------------------+------------------------------------------------------------------------------+


## Saving Standard Name Table

Standard name tables should be saved as xml-documents or yaml-files:

In [14]:
xml_filename = h5tbx.generate_temporary_filename(suffix='.xml')
sc.to_xml(xml_filename)

yml_filename = h5tbx.generate_temporary_filename(suffix='.yml')
sc.to_yaml(yml_filename)
pass

For later usage from anywhere, the table can be registerd with the toolbox. Call `register()`. It save the convention as yml file in the user directory for standard name data and will use the `versionname`:

In [15]:
sc.register(overwrite=True)
StandardNameTable.print_registered()

2023-05-06_19:14:50,127 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com
2023-05-06_19:14:50,127 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com


 > fluid-v1
 > piv-v1
 > Test-v1
 > Test_SNC-v1


Use the command line call

In [16]:
! h5tbx standard_name --list-registered

 > fluid-v1
 > piv-v1
 > Test-v1
 > Test_SNC-v1


2023-05-06_19:14:56,473 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com
2023-05-06_19:14:56,473 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com


## Load Standard Name Convention from file

If you have standard name tables to your hand, just load them. They must be provided as XML or YML:

In [17]:
sc_test = StandardNameTable.from_yaml(yml_filename)
print(sc_test.versionname)
sc_test.dump()

2023-05-06_19:14:56,719 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com
2023-05-06_19:14:56,719 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com


Test_SNC-v1


,canonical_units,description
time,s,physical time
x_velocity,m/s,velocity is a vector quantity. x indicates the component in y-axis direction
y_velocity,m/s,velocity is a vector quantity. y indicates the component in y-axis direction
z_velocity,m/s,velocity is a vector quantity. z indicates the component in z-axis direction


In [18]:
sc_test['x_velocity']

<StandardName: x_velocity [m/s] | SNT: Test_SNC | desc: velocity is a vector quantity. x indicates the component in y-axis direction>

In [19]:
sc_test['x_velocity'].snt

Test_SNC (version number: 1)

Load a registered Standard Name Table from the toolbox:

In [20]:
StandardNameTable.load_registered('Test_SNC-v1').dump()

2023-05-06_19:14:56,783 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com
2023-05-06_19:14:56,783 ERROR    [contact.py:44] Not an ORCID ID: contact@python.com


,canonical_units,description
time,s,physical time
x_velocity,m/s,velocity is a vector quantity. x indicates the component in y-axis direction
y_velocity,m/s,velocity is a vector quantity. y indicates the component in y-axis direction
z_velocity,m/s,velocity is a vector quantity. z indicates the component in z-axis direction


## Load from web
Optimally a community has defined a naming conventions, just like the cfconventions from where the concept is adoped. Let's imort their latest xml document:

In [21]:
cf = StandardNameTable.from_web(url='https://cfconventions.org/Data/cf-standard-names/79/src/cf-standard-name-table.xml')
cf

ImportError: Package "pooch" is needed to download the file cf-standard-name-table.xml

In [ ]:
cf.versionname

In [ ]:
cf.dump(max_rows=4)

## Perform checks
A naming convention can be used to test new standard names, whether they comply with it or not:

In [ ]:
cf.check_name('zenith_angle', strict=True)

In [ ]:
cf['x_wind_gust'].canonical_units

In [ ]:
try:
    cf.check_units('x_wind_gust', units='m/s')
except h5tbx.erros.StandardizedNameError as e:
    print(e)

In [ ]:
try:
    cf.check_units('zenith_angle', units='K')
except h5tbx.conventions.standard_name.StandardNameError as e:
    print(e)
cf.check_units('zenith_angle', units='degree')

Perform a check on a file

In [ ]:
with h5tbx.File(standard_name_table=cf) as h5:
    h5.create_dataset('zenith angle', shape=(3,), units='degree', standard_name='zenith_angle')
    try:
        h5.create_dataset('zenith angle 2', shape=(3,), units='K', standard_name='zenith_angle')
    except h5tbx.conventions.standard_name.StandardNameError as e:
        print(e)        

In [ ]:
cf.check_file(h5.hdf_filename, raise_error=False)

Use the command line call

In [ ]:
cf.register(overwrite=True)

In [ ]:
! h5tbx standard_name -f {h5.hdf_filename} -t {cf.versionname}